# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: Since students are discrete individual units (there can not be a partial student), this is a classification problem.**

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [3]:
# Import libraries
from __future__ import division
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [8]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1]

# TODO: Calculate passing students
n_passed = len(student_data[student_data.passed =="yes"])

# TODO: Calculate failing students
n_failed = len(student_data[student_data.passed =="no"])

# TODO: Calculate graduation rate
grad_rate = n_passed / (n_passed + n_failed) * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 31
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [6]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [7]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [11]:
# Import any additional functionality you may need here
num_all = X_all.shape[0]
# Set the number of training points
num_train = 300
# Set the number of testing points
num_test = num_all - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = num_test)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

I'll be trying the following three sklearn learning models: DecisionTreeClassifier, KNeighborsClassifier and GaussianNB.  I selected these methods because they are the ones that are the most mathematically intuitive to me, rather than the ones that would necessarily give the best results.  Additionally, I wrote a "Random" classifier, for practice and comparison.  Classifier choice takes into account size of data set and the bias and variance of that data set. Our data has ?variance and ?bias

**Random**

I wanted to understand better how to code classifiers so I added code for a random classifier.  I expect this classifier to get it right about half the time.  This is about what I found, though the results changed wildly each time I ran it!  Certainly the other three classifiers in this exercise will do much better than the random one!

**Decision Tree**

Using a decision tree to classify outcomes is the easiest way to fathom and visualize a learning model.  Having been introduced to this concept earlier with the Titanic and Iris data, I was delighted to interact with it again on another dataset as a baseline. 

Decision Trees appear to be the fastest classifier to train and test data with. 

Meanwhile, Decisions trees assume the problem is linearly separable, meaning they perform poorly on radially symmetric input data. It's easy to overfit a decision tree, only to find later that it made poor predictions. It seems like it's worse if there is too much data, because as the complexity increases, it's like you're trying to force a square peg into a potentially round hole, fitwise. What that means in practice is that is too many minute subdivisions.     


**K Nearest Neighbors**

The most famous application of KNN is when recommending that a customer buy an item based on what other customers have purchased who share a similar buying history to that customer.  

KNN is a good choice when you don't know much about your underlying distribution of the dataset. It has a polynomial decision boundary.  Such a boundary is flexible (moreso than Naive Bayes' which can only be parabolic or linear) and so for the right datasets, will be able to actually fit them. 

A weakness of KNN are when there are too many dimensions involved and it's hard for a useful distance measure to precipitate. Further, each attribute is given equal weight in the distance calculation, which can be problematic for irrelevant attributes. For this data set I feel like we have a wieldly number of dimensions which gives me faith that KNN will not become overly biased and performance wont be an issue. There is a concern that KNN will overfit the data since this method is susceptible to errors from bias when using small samples.  So I'm curious to see how it compares to the other classifiers on this fairly small dataset. 

I didn't modify any parameters, so the results you see below are with the default settings. I understand that KNN will have a range of N values that are better than others.  


**Naive Bayes Gaussian**

Naive Bayes is a goto classifier for small datasets like the one student intervention dataset.  Since I had also selected KNN for a classifier, I wanted to compare it with NB, as both are purported to perform well on smaller, labelled datasets.  This 

Bayes is all about knowing prior probabilities.  The classic example of Bayes' theorem is predicting how likely it is that a patient has cancer given the results of their test given the chance in the population at large.   Bayes' theorem gives rise to the concepts of sensitivity vs specificity.   

A weakness of NB is that it treats features as independence of each other.  Unfortunately, I suspect that many features of the students' lives are dependant such as for example the student's zipcode and the employment status of their parents. 

I still think that the results of the NB model will be informative.   




In [53]:
import random

class RandomClassifier():
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
    
    def predict(self, X_test):
        predictions = [random.choice(["no", "yes"]) for i in range(len(X_test))]
        return predictions

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [46]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [55]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from sklearn import model_C

# TODO: Initialize the three models
clf_A = DecisionTreeClassifier()
clf_B = KNeighborsClassifier()
clf_C = GaussianNB()
clf_R = RandomClassifier()


# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

train_predict(clf_R, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_R, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_R, X_train_300, y_train_300, X_test, y_test)

#train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
# train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
# train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)

# train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
# train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
# train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)

# train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
# train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
# train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)



# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

Training a RandomClassifier using a training set size of 100. . .
Trained model in 0.0000 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 0.6000.
Made predictions in 0.0001 seconds.
F1 score for test set: 0.5310.
Training a RandomClassifier using a training set size of 200. . .
Trained model in 0.0000 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 0.5333.
Made predictions in 0.0001 seconds.
F1 score for test set: 0.6239.
Training a RandomClassifier using a training set size of 300. . .
Trained model in 0.0000 seconds
Made predictions in 0.0001 seconds.
F1 score for training set: 0.5810.
Made predictions in 0.0001 seconds.
F1 score for test set: 0.5421.


### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 0 - RandomClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |          0.0000               |    0.0002                    |     0.6000             |             0.5310   |
| 200               |        0.0000          |          0.0002              |           0.6000       |   0.6239              |
| 300               |     0.0000                    |        0.0001                |      0.5333            |    0.5421      |

** Classifer 1 - DecisionTreeClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |          0.0008               |    0.0002                    |     1.0000             |              0.7176   |
| 200               |        0.0009          |          0.0002              |           1.0000       |   0.6446              |
| 300               |     0.0020                    |        0.0003                |      1.0000            |    0.7087      |

** Classifer 2 - KNeighborsClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0008                  |     0.0009                   |      0.8188            |            0.8310     |
| 200               |     0.0008             |             0.0031           |              0.8316    |       0.8345          |
| 300               |      0.0007                   |       0.0044                 |         0.8494         |     0.8112     |

** Classifer 3 - GaussianNB**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0010                   |  0.0002                      |       0.7752           |   0.7273              |
| 200               |      0.0012                   |   0.0005                     |        0.8159          |     0.8030            |
| 300               |      0.0013                   |   0.0006                     |         0.7665         |   0.7576              |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** Of the four models I examined, the KNN Classifier gave the best F1 score for our dataset.  While not a perfect indicator of student success it will help us target at-risk students so they can be offered more resources to bridge the gap to graduation.   

Since we have a relatively small student body and small set of data, the KNN algorithm is the best we can do --in a sense comparing our at-risk students to their peers.  High schoolers often exhibit pack behavior, which intuitively lends creedence to a clusterful analysis.  

Of note is that as we add additional students to our analysis, we have to make N comparisons, where N is the number of existing students.  At present, the quantities in play are reasonable and wont tax our technology infrastructure too much.  That is to say, this test is relatively inexpensive.   

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: ** KNN looks at each attribute of a student and compares the closeness of this attribute to other students.   For example, a student with 5 absenses is closer to a student with 6 absenses than to a student who was absent twice.  But we are looking at more than just attendance.   We're looking at family size, age, parental income and so on. KNN is a way of taking into account all these attributes of a student at the same time and using that to give an indication of whether a student will graduate.  In essence, KNN feels like the old addage "if it looks like a duck, and it quacks like a duck...it is a duck".  More specifically, we investigated the outcomes of many students to train our model, so that future students can be compared to them.  

Education changes over time --some school districts acquire funding for magnet schools, some neighborhoods take economic slides, and some subjects, STEM for example, have become more popular for students to study.  All of these factors will have an impact on training our model and so care must be taken to retrain the model with contemporary data.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [73]:
# TODO: Import 'GridSearchCV' and 'make_scorer'

from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
parameters = {
    'n_neighbors': [1,3,5,7,9],
    'weights': ['distance', 'uniform'],
    'algorithm': ['ball_tree', 'kd_tree', 'brute', 'auto'],
    'leaf_size': [1,5,30,100],
    'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski'],
}

# TODO: Initialize the classifier
clf = KNeighborsClassifier()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
#grid_obj = None
grid_obj.fit(X_train, y_train)
# Get the estimator
clf = grid_obj.best_estimator_

print(clf)
# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

KNeighborsClassifier(algorithm='ball_tree', leaf_size=1, metric='manhattan',
           metric_params=None, n_jobs=1, n_neighbors=9, p=2,
           weights='distance')
Made predictions in 0.0091 seconds.
Tuned model has a training F1 score of 1.0000.
Made predictions in 0.0031 seconds.
Tuned model has a testing F1 score of 0.7703.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** The untuned model had scores 0.8316(trained) and 0.8345(tested).  The tuned model is interesting, with a training score of 1.000 and testing score of 0.7703 it appears to be a severe overfit. I question the priorities of the "best_estimator" function as it appears to blindly look at the training score without taking into account the context of the testing score.  Something to watch out for in the future.  I suspect that the model can be tuned to do better for both training and testing than the untuned model, I just didn't happen upon the particular parameters using the methods in this assignment.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.